In [3]:
import pandas as pd
file_path = 'tofdrug2_cohortdate.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_36764\1951161746.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='ISO-8859-1')


In [7]:
import re

# 약물 이름에서 용량과 관련된 정보를 제거하는 함수
def simplify_drug_name(name):
    # 용량, 용기, 제형 등을 제거
    simplified_name = re.sub(r'\s+\d*\.?\d*\s*(mg|ml|mEq|g|%|L|unit)(/\d*\.?\d*\s*(mg|ml|mEq|g|%|L|unit))?|\d*\.?\d*unit/\w+|\s+\d+\s*|\s+tab|\s+btl|\s+inj|\s+nebule|\s+bag', '', name, flags=re.I)
    simplified_name = re.sub(r'[,*]', '', simplified_name).strip()
    return simplified_name.strip()

# 다음으로, df에 simplified_drug_name 컬럼을 추가합니다.
df['simplified_drug_name'] = df['source_name'].apply(simplify_drug_name)

In [9]:
# 날짜 컬럼의 데이터 타입을 datetime으로 변환
df['cohort_start_date'] = pd.to_datetime(df['cohort_start_date'])
df['cohort_end_date'] = pd.to_datetime(df['cohort_end_date'])
df['drug_exposure_start_date'] = pd.to_datetime(df['drug_exposure_start_date'])
df['drug_exposure_end_date'] = pd.to_datetime(df['drug_exposure_end_date'])

# cohort 기간 내의 drug exposure 데이터 필터링
within_cohort = df[(df['drug_exposure_start_date'] >= df['cohort_start_date']) & 
                   (df['drug_exposure_end_date'] <= df['cohort_end_date'])]

# simplified_drug_name에 대한 처방 횟수 계산
drug_counts = within_cohort['simplified_drug_name'].value_counts().reset_index()
drug_counts.columns = ['simplified_drug_name', 'count']

# 전체 환자 수 계산
total_patients = df['person_id'].nunique()

# 각 약물을 처방받은 환자의 중복을 제거
unique_prescriptions = within_cohort[['person_id', 'simplified_drug_name']].drop_duplicates()

# 고유 환자 수를 기준으로 각 약물에 대한 카운트 계산
unique_drug_counts = unique_prescriptions.groupby('simplified_drug_name').size().reset_index(name='count')

# 처방받은 환자의 비율 계산
unique_drug_counts['percentage'] = (unique_drug_counts['count'] / total_patients) * 100

# 가장 많이 처방된 약물의 Top 10 찾기
top_10_unique_drugs = unique_drug_counts.sort_values(by='count', ascending=False).head(10)

top_10_unique_drugs


,simplified_drug_name,count,percentage
701,Normal saline,1318,84.922680
451,Hartmann,1248,80.412371
294,Dextrose,1243,80.090206
639,Midazolam,1194,76.932990
423,Furosemide,1152,74.226804
368,Famotidine,1139,73.389175
455,Heparin sodium,1132,72.938144
48,Ambroxol hydrochloride,1130,72.809278
772,Plasma solution A,1129,72.744845
907,Spironolactone,1108,71.391753


In [18]:
# 무작위로 10개 행 출력
within_cohort_sample = within_cohort.sample(10, random_state=1)
selected_columns_sample = within_cohort_sample[['person_id', 'cohort_start_date', 'cohort_end_date', 'drug_exposure_start_date', 'drug_exposure_end_date']]
selected_columns_sample


,person_id,cohort_start_date,cohort_end_date,drug_exposure_start_date,drug_exposure_end_date
46030,679569,2006-11-07,2022-01-26,2009-06-23,2009-06-23
281464,2752528,2019-07-14,2022-05-10,2019-10-28,2019-10-28
221849,2625117,2015-11-28,2022-03-17,2015-12-28,2015-12-28
171274,2620168,2015-10-12,2022-09-06,2015-11-10,2015-11-10
503393,2192009,2005-12-25,2022-08-18,2015-03-20,2015-03-20
633704,2399646,2009-10-13,2022-01-13,2010-02-10,2010-02-10
419247,2248140,2004-11-15,2022-07-13,2006-05-04,2006-05-04
290986,2756092,2019-08-28,2022-09-29,2019-12-31,2019-12-31
387456,1919555,2017-12-18,2022-02-03,2017-12-20,2017-12-20
214975,2512177,2012-12-14,2022-09-19,2013-03-24,2013-03-24


In [13]:
total_patients

1552

In [14]:
# 각 약물에 대한 성별별 환자 수 계산
gender_drug_counts = within_cohort.drop_duplicates(subset=['person_id', 'simplified_drug_name', 'gender_source_value'])
gender_drug_counts = gender_drug_counts.groupby(['simplified_drug_name', 'gender_source_value']).size().unstack(fill_value=0)

# Top 10 약물 목록 준비 (이전 단계에서 계산된 top_10_unique_drugs 변수 사용)
top_10_drugs_list = top_10_unique_drugs['simplified_drug_name'].tolist()

# Top 10 약물에 대한 성별별 환자 수
top_10_gender_counts = gender_drug_counts.loc[top_10_drugs_list]

# 전체 환자 수 계산
total_male_patients = df[df['gender_source_value'] == 'M']['person_id'].nunique()
total_female_patients = df[df['gender_source_value'] == 'F']['person_id'].nunique()

# 각 약물별로 남성과 여성에게 처방된 비율 계산
top_10_gender_counts['male_percentage'] = (top_10_gender_counts['M'] / total_male_patients) * 100
top_10_gender_counts['female_percentage'] = (top_10_gender_counts['F'] / total_female_patients) * 100

top_10_gender_counts[['male_percentage', 'female_percentage']]


gender_source_value,male_percentage,female_percentage
simplified_drug_name,,
Normal saline,84.346847,85.692771
Hartmann,80.405405,80.421687
Dextrose,79.054054,81.475904
Midazolam,77.139640,76.656627
Furosemide,73.536036,75.150602
Famotidine,74.099099,72.439759
Heparin sodium,72.860360,73.042169
Ambroxol hydrochloride,73.423423,71.987952
Plasma solution A,73.310811,71.987952


In [17]:
# 전체 환자 수 대비 각 성별의 비율을 계산하기 위한 코드 수정
final_table_with_percentages = top_10_gender_counts.copy()
final_table_with_percentages['male_percentage_of_total'] = (final_table_with_percentages['M'] / final_table_with_percentages['total_patients']) * 100
final_table_with_percentages['female_percentage_of_total'] = (final_table_with_percentages['F'] / final_table_with_percentages['total_patients']) * 100

# 컬럼명을 요구사항에 맞게 조정
final_table_adjusted = final_table_with_percentages.rename(columns={
    'M': 'male_count',
    'F': 'female_count',
    'male_percentage_of_total': 'male_count, %',
    'female_percentage_of_total': 'female_count, %',
    'total_patients': 'total_count'
})

# 필요한 컬럼만 선택하여 최종 표 형식을 맞춤
final_table_adjusted = final_table_adjusted[['male_count', 'male_count, %', 'female_count', 'female_count, %', 'total_count']]

final_table_adjusted


gender_source_value,male_count,"male_count, %",female_count,"female_count, %",total_count
simplified_drug_name,,,,,
Normal saline,749,56.828528,569,43.171472,1318
Hartmann,714,57.211538,534,42.788462,1248
Dextrose,702,56.476267,541,43.523733,1243
Midazolam,685,57.370184,509,42.629816,1194
Furosemide,653,56.684028,499,43.315972,1152
Famotidine,658,57.769974,481,42.230026,1139
Heparin sodium,647,57.155477,485,42.844523,1132
Ambroxol hydrochloride,652,57.699115,478,42.300885,1130
Plasma solution A,651,57.661647,478,42.338353,1129


In [19]:
# df 데이터프레임에서 남성과 여성의 수 계산
male_female_counts = df.drop_duplicates(subset=['person_id'])['gender_source_value'].value_counts()

male_female_counts


gender_source_value
M    888
F    664
Name: count, dtype: int64